In [7]:
import requests
import datetime
import bs4
import numpy as np
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [358]:
urls = ['https://www.cricbuzz.com/cricket-series/4061/indian-premier-league-2022/matches',
'https://www.cricbuzz.com/cricket-series/3472/indian-premier-league-2021/matches',
'https://www.cricbuzz.com/cricket-series/3130/indian-premier-league-2020/matches',
'https://www.cricbuzz.com/cricket-series/2810/indian-premier-league-2019/matches',
'https://www.cricbuzz.com/cricket-series/2676/indian-premier-league-2018/matches',
'https://www.cricbuzz.com/cricket-series/2568/indian-premier-league-2017/matches',
'https://www.cricbuzz.com/cricket-series/2430/indian-premier-league-2016/matches',
'https://www.cricbuzz.com/cricket-series/2330/indian-premier-league-2015/matches',
'https://www.cricbuzz.com/cricket-series/2261/indian-premier-league-2014/matches',
'https://www.cricbuzz.com/cricket-series/2170/indian-premier-league-2013/matches',
'https://www.cricbuzz.com/cricket-series/2115/indian-premier-league-2012/matches',
'https://www.cricbuzz.com/cricket-series/2037/indian-premier-league-2011/matches',
'https://www.cricbuzz.com/cricket-series/2060/indian-premier-league-2010/matches',
'https://www.cricbuzz.com/cricket-series/2059/indian-premier-league-2009/matches',
'https://www.cricbuzz.com/cricket-series/2058/indian-premier-league-2008/matches']

In [8]:
url_2022 = 'https://www.cricbuzz.com/cricket-series/4061/indian-premier-league-2022/matches'
url_2021 = 'https://www.cricbuzz.com/cricket-series/3472/indian-premier-league-2021/matches'
url_2020 = 'https://www.cricbuzz.com/cricket-series/3130/indian-premier-league-2020/matches'
url_2019 = 'https://www.cricbuzz.com/cricket-series/2810/indian-premier-league-2019/matches'
url_2018 = 'https://www.cricbuzz.com/cricket-series/2676/indian-premier-league-2018/matches'
url_2017 = 'https://www.cricbuzz.com/cricket-series/2568/indian-premier-league-2017/matches'
url_2016 = 'https://www.cricbuzz.com/cricket-series/2430/indian-premier-league-2016/matches'
url_2015 = 'https://www.cricbuzz.com/cricket-series/2330/indian-premier-league-2015/matches'
url_2014 = 'https://www.cricbuzz.com/cricket-series/2261/indian-premier-league-2014/matches'
url_2013 = 'https://www.cricbuzz.com/cricket-series/2170/indian-premier-league-2013/matches'
url_2012 = 'https://www.cricbuzz.com/cricket-series/2115/indian-premier-league-2012/matches'
url_2011 = 'https://www.cricbuzz.com/cricket-series/2037/indian-premier-league-2011/matches'
url_2010 = 'https://www.cricbuzz.com/cricket-series/2060/indian-premier-league-2010/matches'
url_2009 = 'https://www.cricbuzz.com/cricket-series/2059/indian-premier-league-2009/matches'
url_2008 = 'https://www.cricbuzz.com/cricket-series/2058/indian-premier-league-2008/matches'

In [61]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver,10)
driver.get(url_2009)
driver.maximize_window()
sleep(3)

In [10]:
matches = []
matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-brdr cb-series-matches ng-scope']"))
matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-matches ng-scope']"))

match_links = []
for i in matches:
    match_links.append(i.find_element(By.XPATH,".//a").get_attribute('href'))

In [11]:
len(match_links)

59

In [12]:
def out(i):
    if ' b ' in i:
         i =(i.split(' b ')[1])
    elif 'run out' in i:
        i = ('run out')
    elif 'not out' in i:
        i = ('not out')
    return i


def winner(i):
    if i == won:
        j = 'won'
    else:
        j = 'lost'
    return j

In [13]:
def year(i):
    return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).year 

def month(i):
    return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).month  

def day(i):
    return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).day  

def time(i):
    return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).time()  

# Batters

In [352]:
i=10

In [353]:
"{}{}".format('url_20',i)

'url_2010'

In [368]:
def get_batters_data(url):
    driver.get(url)
    sleep(2)
    matches = []
    matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-brdr cb-series-matches ng-scope']"))
    matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-matches ng-scope']"))

    match_links = []
    for i in matches:
        match_links.append(i.find_element(By.XPATH,".//a").get_attribute('href'))

    cols = ['batsman', 'got_out', 'Runs', 'Balls', 'Fours', 'Sixes', 'St.rate',
           'Inning', 'Team', 'Opponent', 'Win/Lost', 'match_id', 'time', 'day',
           'year', 'Month', 'Venue']
    Batters_data = pd.DataFrame(columns=cols)
    for i in match_links:
        try:
            a = i
            match_id = a.split('/')[4]
            b = a.split('/')
            b.insert(3,'live-cricket-scorecard')
            b.pop(4)
            match = "/".join(b)
            data = requests.get(match)

            soup = bs4.BeautifulSoup(data.content,'html.parser')
            # winning team
            won = soup.find('div',"cb-col cb-scrcrd-status cb-col-100 cb-text-complete").text.split('won')[0].strip()
            # timestamp
            timestamp = soup.find_all('div','cb-col cb-col-73')[2].find('span').get('timestamp')
            # Venue
            venue = soup.find_all('div','cb-col cb-col-73')[3].text.strip()
            def year(i):
                return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).year 

            def month(i):
                return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).month  

            def day(i):
                return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).day  

            def time(i):
                return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).time()  
            day = day(timestamp)
            month = month(timestamp)
            year = year(timestamp)
            time = time(timestamp)

            #########                Inning 1           ###########

            inn1 = soup.find('div',id='innings_1')
            inn2 = soup.find('div',id='innings_2')
            t2 = inn2.find_all('div','cb-col cb-col-100 cb-scrd-hdr-rw')[0].text.split('Innings')[0].strip()
            t1 = inn1.find_all('div','cb-col cb-col-100 cb-scrd-hdr-rw')[0].text.split('Innings')[0].strip()

            batters = inn1.find_all('div','cb-col cb-col-100 cb-ltst-wgt-hdr')[0]

            batsman = []
            ID = []
            for i in batters1.find_all('div','cb-col cb-col-100 cb-scrd-itms')[:-2]:
                batsman.append(i.text.split('  ')[1].strip())
                try:
                    ID.append(i.find('a').get('href').split('/')[2].strip())
                except:
                    ID.append(np.nan)
            got_out = []
            for i in batters.find_all('div','cb-col cb-col-33')[1:]:
                got_out.append(i.text.strip())

            runs = []
            for i in batters.find_all('div','cb-col cb-col-8 text-right text-bold')[1:]:
                runs.append(i.text.strip())

            df = pd.DataFrame([batsman,ID,got_out,runs]).T
            df.columns = ['batsman',"ID",'got_out','Runs']

            x = []
            for i in batters.find_all('div','cb-col cb-col-8 text-right')[4:]:
                x.append(i.text.strip())   
            n = 4
            out = [x[k:k+n] for k in range(0, len(x), n)]

            y = pd.DataFrame(out,columns=['Balls','Fours','Sixes','St.rate'])
            y['batsman'] = pd.Series(batsman)
            df1 = pd.merge(df,y,on='batsman',how='inner')


            team = [t1 for i in range(df1.shape[0])]
            df1['Inning'] = ['Inn1' for i in range(df1.shape[0])]
            df1['Team'] = team
            df1['Opponent'] = [t2 for i in range(df1.shape[0])]
    #         df1['ID'] = ID


            # df['Out']=df.got_out.apply(out)

            #####             Inning 2             #####

            batters = inn2.find_all('div','cb-col cb-col-100 cb-ltst-wgt-hdr')[0]

            batsman = []
            ID = []
            for i in batters.find_all('div','cb-col cb-col-100 cb-scrd-itms')[:-2]:
                batsman.append(i.text.split('  ')[1].strip())
                try:
                    ID.append(i.find('a').get('href').split('/')[2].strip())
                except:
                    ID.append(np.nan)
            got_out = []
            for i in batters2.find_all('div','cb-col cb-col-33')[1:]:
                got_out.append(i.text.strip())

            runs = []
            for i in batters.find_all('div','cb-col cb-col-8 text-right text-bold')[1:]:
                runs.append(i.text.strip())

            df = pd.DataFrame([batsman,ID,got_out,runs]).T
            df.columns = ['batsman',"ID",'got_out','Runs']

            x = []
            for i in batters.find_all('div','cb-col cb-col-8 text-right')[4:]:
                x.append(i.text.strip())   
            n = 4
            out = [x[k:k+n] for k in range(0, len(x), n)]

            y = pd.DataFrame(out,columns=['Balls','Fours','Sixes','St.rate'])
            y['batsman'] = pd.Series(batsman)
            df2 = pd.merge(df,y,on='batsman',how='inner')


            team = [t2 for i in range(df2.shape[0])]
            df2['Inning'] = ['Inn2' for i in range(df2.shape[0])]
            df2['Team'] = team
            df2['Opponent'] = [t1 for i in range(df2.shape[0])]
    #         df2['ID'] = ID

            # df['Out']=df.got_out.apply(out)

            df_new = pd.concat([df1,df2],ignore_index=True)
            df_new['Win/Lost'] = df_new.Team.apply(winner)
            df_new['match_id'] = [match_id for i in range(df_new.shape[0])]
            df_new['time'] =  [time for i in range(df_new.shape[0])]
            df_new['day'] = [day for i in range(df_new.shape[0])]
            df_new['year'] =  [year for i in range(df_new.shape[0])]
            df_new['Month'] =  [month for i in range(df_new.shape[0])]
            df_new['Venue'] =  [venue for i in range(df_new.shape[0])]
            Batters_data = pd.concat([Batters_data,df_new],ignore_index=True)
        except:
            print(match)
            pass
    return Batters_data

In [371]:
for i in urls[:-3]:
    df = get_batters_data(i)
    Batters_data = pd.concat([Batters_data,df],ignore_index=True)

https://www.cricbuzz.com/live-cricket-scorecard/35718/kkr-vs-rcb-30th-match-indian-premier-league-2021
https://www.cricbuzz.com/live-cricket-scorecard/35723/srh-vs-mi-31st-match-indian-premier-league-2021
https://www.cricbuzz.com/live-cricket-scorecard/18149/rcb-vs-srh-29th-match-indian-premier-league-2017
https://www.cricbuzz.com/live-cricket-scorecard/14619/kkr-vs-rr-25th-match-indian-premier-league-2015
https://www.cricbuzz.com/live-cricket-scorecard/14623/rcb-vs-rr-29th-match-indian-premier-league-2015
https://www.cricbuzz.com/live-cricket-scorecard/11924/pbks-vs-srh-59th-match-indian-premier-league-2013
https://www.cricbuzz.com/live-cricket-scorecard/11213/kkr-vs-dcg-32nd-match-indian-premier-league-2012
https://www.cricbuzz.com/live-cricket-scorecard/11215/rcb-vs-csk-34th-match-indian-premier-league-2012
https://www.cricbuzz.com/live-cricket-scorecard/9917/rcb-vs-rr-20th-match-indian-premier-league-2011
https://www.cricbuzz.com/live-cricket-scorecard/9966/dc-vs-pwi-68th-match-ind

In [373]:
Batters_data.to_csv('All_batters.csv',index=False)

# Bowlers

In [149]:
# 2008 to 2016

driver.get(url_2022)
matches = []
matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-brdr cb-series-matches ng-scope']"))
matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-matches ng-scope']"))

match_links = []
for i in matches:
    match_links.append(i.find_element(By.XPATH,".//a").get_attribute('href'))
cols = ['bowler', 'ID', 'overs', 'maidens', 'runs', 'wickets', 'no_balls',
       'wides', 'economy', 'team', 'opponent', 'Inning', 'Win/Lost',
       'match_id', 'time', 'day', 'year', 'Month', 'Venue']
bowlers = pd.DataFrame(columns=cols)

for i in match_links:
    try:
        bowler = []
        ID = []
        overs = []
        meidens = []
        runs = []
        wkts = []
        no_balls = []
        wides = []
        eco = []
        a  = i
        match_id = a.split('/')[4]
        b = a.split('/')
        b.insert(3,'live-cricket-scorecard')
        b.pop(4)
        match = "/".join(b)
        data = requests.get(match)
        soup = bs4.BeautifulSoup(data.content,'html.parser')
        inn1 = soup.find('div',id='innings_1')
        inn2 = soup.find('div',id='innings_2')
        t2 = inn2.find_all('div','cb-col cb-col-100 cb-scrd-hdr-rw')[0].text.split('Innings')[0].strip()
        t1 = inn1.find_all('div','cb-col cb-col-100 cb-scrd-hdr-rw')[0].text.split('Innings')[0].strip()
        bowl2 = inn2.find_all('div','cb-col cb-col-100 cb-ltst-wgt-hdr')[-1]
        bowl1 = inn1.find_all('div','cb-col cb-col-100 cb-ltst-wgt-hdr')[-1]

        won = soup.find('div',"cb-col cb-scrcrd-status cb-col-100 cb-text-complete").text.split('won')[0].strip()
        # timestamp
        timestamp = soup.find_all('div','cb-col cb-col-73')[2].find('span').get('timestamp')
        # Venue
        venue = soup.find_all('div','cb-col cb-col-73')[3].text.strip()
        def year(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).year 

        def month(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).month  

        def day(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).day  

        def time(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).time()  
        day = day(timestamp)
        month = month(timestamp)
        year = year(timestamp)
        time = time(timestamp)

        for i in bowl1.find_all('div','cb-col cb-col-100 cb-scrd-itms'):
            bowler.append(i.find_all('div')[0].text.strip())
            ID.append(i.find_all('div')[0].find('a').get('href').split('/')[2])
            overs.append(i.find_all('div')[1].text.strip())
            meidens.append(i.find_all('div')[2].text.strip())
            runs.append(i.find_all('div')[3].text.strip())
            wkts.append(i.find_all('div')[4].text.strip())
            no_balls.append(i.find_all('div')[5].text.strip())
            wides.append(i.find_all('div')[6].text.strip())
            eco.append(i.find_all('div')[7].text.strip())

        df1 = pd.DataFrame({
            'bowler':bowler,
            'ID':ID,
            'overs':overs,
            'maidens':meidens,
            'runs':runs,
            'wickets':wkts,
            'no_balls':no_balls,
            'wides':wides,
            'economy':eco
        })

        df1['team'] = [t2 for i in range(df1.shape[0])]
        df1['opponent'] = [t1 for i in range(df1.shape[0])]
        df1['Inning'] = ['Inn1' for i in range(df1.shape[0])]
        bowler = []
        ID = []
        overs = []
        meidens = []
        runs = []
        wkts = []
        no_balls = []
        wides = []
        eco = []


        for i in bowl2.find_all('div','cb-col cb-col-100 cb-scrd-itms'):
            bowler.append(i.find_all('div')[0].text.strip())
            ID.append(i.find_all('div')[0].find('a').get('href').split('/')[2])
            overs.append(i.find_all('div')[1].text.strip())
            meidens.append(i.find_all('div')[2].text.strip())
            runs.append(i.find_all('div')[3].text.strip())
            wkts.append(i.find_all('div')[4].text.strip())
            no_balls.append(i.find_all('div')[5].text.strip())
            wides.append(i.find_all('div')[6].text.strip())
            eco.append(i.find_all('div')[7].text.strip())

        df2 = pd.DataFrame({
            'bowler':bowler,
            'ID':ID,
            'overs':overs,
            'maidens':meidens,
            'runs':runs,
            'wickets':wkts,
            'no_balls':no_balls,
            'wides':wides,
            'economy':eco
        })

        df2['team'] = [t1 for i in range(df2.shape[0])]
        df2['opponent'] = [t2 for i in range(df2.shape[0])]
        df2['Inning'] = ['Inn2' for i in range(df2.shape[0])]
        df_new = pd.concat([df1,df2],ignore_index=True)
        df_new = pd.concat([df1,df2],ignore_index=True)
        df_new['Win/Lost'] = df_new.team.apply(winner)
        df_new['match_id'] = [match_id for i in range(df_new.shape[0])]
        df_new['time'] =  [time for i in range(df_new.shape[0])]
        df_new['day'] = [day for i in range(df_new.shape[0])]
        df_new['year'] =  [year for i in range(df_new.shape[0])]
        df_new['Month'] =  [month for i in range(df_new.shape[0])]
        df_new['Venue'] =  [venue for i in range(df_new.shape[0])]
        bowlers = pd.concat([bowlers,df_new],ignore_index=True)
    except:
        print(match)
        pass

https://www.cricbuzz.com/live-cricket-scorecard/18149/rcb-vs-srh-29th-match-indian-premier-league-2017


In [298]:
# 2017 to 2022

driver.get(url_2022)
matches = []
matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-brdr cb-series-matches ng-scope']"))
matches.extend(driver.find_elements(By.XPATH,"//div[@class='cb-col-100 cb-col cb-series-matches ng-scope']"))

match_links = []
for i in matches:
    match_links.append(i.find_element(By.XPATH,".//a").get_attribute('href'))
cols = ['bowler', 'ID', 'overs', 'maidens', 'runs', 'wickets', 'no_balls',
       'wides', 'economy', 'team', 'opponent', 'Inning', 'Win/Lost',
       'match_id', 'time', 'day', 'year', 'Month', 'Venue']
bowlers = pd.DataFrame(columns=cols)

for i in match_links:
    try:
        bowler = []
        ID = []
        overs = []
        meidens = []
        runs = []
        wkts = []
        no_balls = []
        wides = []
        eco = []
        a  = i
        match_id = a.split('/')[4]
        b = a.split('/')
        b.insert(3,'live-cricket-scorecard')
        b.pop(4)
        match = "/".join(b)
        data = requests.get(match)
        soup = bs4.BeautifulSoup(data.content,'html.parser')
        inn1 = soup.find('div',id='innings_1')
        inn2 = soup.find('div',id='innings_2')
        t2 = inn2.find_all('div','cb-col cb-col-100 cb-scrd-hdr-rw')[0].text.split('Innings')[0].strip()
        t1 = inn1.find_all('div','cb-col cb-col-100 cb-scrd-hdr-rw')[0].text.split('Innings')[0].strip()
        bowl2 = inn2.find_all('div','cb-col cb-col-100 cb-ltst-wgt-hdr')[-2]
        bowl1 = inn1.find_all('div','cb-col cb-col-100 cb-ltst-wgt-hdr')[-2]

        won = soup.find('div',"cb-col cb-scrcrd-status cb-col-100 cb-text-complete").text.split('won')[0].strip()
        # timestamp
        timestamp = soup.find_all('div','cb-col cb-col-73')[2].find('span').get('timestamp')
        # Venue
        venue = soup.find_all('div','cb-col cb-col-73')[3].text.strip()
        def year(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).year 

        def month(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).month  

        def day(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).day  

        def time(i):
            return datetime.datetime.fromtimestamp(int(i)/ 1e3 ).time()  
        day = day(timestamp)
        month = month(timestamp)
        year = year(timestamp)
        time = time(timestamp)

        for i in bowl1.find_all('div','cb-col cb-col-100 cb-scrd-itms'):
            bowler.append(i.find_all('div')[0].text.strip())
            ID.append(i.find_all('div')[0].find('a').get('href').split('/')[2])
            overs.append(i.find_all('div')[1].text.strip())
            meidens.append(i.find_all('div')[2].text.strip())
            runs.append(i.find_all('div')[3].text.strip())
            wkts.append(i.find_all('div')[4].text.strip())
            no_balls.append(i.find_all('div')[5].text.strip())
            wides.append(i.find_all('div')[6].text.strip())
            eco.append(i.find_all('div')[7].text.strip())

        df1 = pd.DataFrame({
            'bowler':bowler,
            'ID':ID,
            'overs':overs,
            'maidens':meidens,
            'runs':runs,
            'wickets':wkts,
            'no_balls':no_balls,
            'wides':wides,
            'economy':eco
        })

        df1['team'] = [t2 for i in range(df1.shape[0])]
        df1['opponent'] = [t1 for i in range(df1.shape[0])]
        df1['Inning'] = ['Inn1' for i in range(df1.shape[0])]
        bowler = []
        ID = []
        overs = []
        meidens = []
        runs = []
        wkts = []
        no_balls = []
        wides = []
        eco = []


        for i in bowl2.find_all('div','cb-col cb-col-100 cb-scrd-itms'):
            bowler.append(i.find_all('div')[0].text.strip())
            ID.append(i.find_all('div')[0].find('a').get('href').split('/')[2])
            overs.append(i.find_all('div')[1].text.strip())
            meidens.append(i.find_all('div')[2].text.strip())
            runs.append(i.find_all('div')[3].text.strip())
            wkts.append(i.find_all('div')[4].text.strip())
            no_balls.append(i.find_all('div')[5].text.strip())
            wides.append(i.find_all('div')[6].text.strip())
            eco.append(i.find_all('div')[7].text.strip())

        df2 = pd.DataFrame({
            'bowler':bowler,
            'ID':ID,
            'overs':overs,
            'maidens':meidens,
            'runs':runs,
            'wickets':wkts,
            'no_balls':no_balls,
            'wides':wides,
            'economy':eco
        })

        df2['team'] = [t1 for i in range(df2.shape[0])]
        df2['opponent'] = [t2 for i in range(df2.shape[0])]
        df2['Inning'] = ['Inn2' for i in range(df2.shape[0])]
        df_new = pd.concat([df1,df2],ignore_index=True)
        df_new = pd.concat([df1,df2],ignore_index=True)
        df_new['Win/Lost'] = df_new.team.apply(winner)
        df_new['match_id'] = [match_id for i in range(df_new.shape[0])]
        df_new['time'] =  [time for i in range(df_new.shape[0])]
        df_new['day'] = [day for i in range(df_new.shape[0])]
        df_new['year'] =  [year for i in range(df_new.shape[0])]
        df_new['Month'] =  [month for i in range(df_new.shape[0])]
        df_new['Venue'] =  [venue for i in range(df_new.shape[0])]
        bowlers = pd.concat([bowlers,df_new],ignore_index=True)
    except:
        print(match)
        pass